In [ ]:
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
!pip install python-libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive
!apt-get -qq install -y graphviz && pip install pydot
import pydot
!pip install cartopy
import cartopy

E: Package 'libfluidsynth1' has no installation candidate
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 14.4 MB/s eta 0:00:00
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 1.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for libarchive
  Running setup.py clean for libarchive
Failed to build libarchive
ERROR: Could not build wheels for libar

In [ ]:
!pip install data
!pip list

  Preparing metadata (setup.py) ... done
  Created wheel for data: filename=data-0.4-py3-none-any.whl size=7228 sha256=ed032ba086cdfbc1251958072aabe87c220da7fafa543fab7358ca3fa6d25e73
  Stored in directory: /root/.cache/pip/wheels/d0/e8/fa/e253c256048ea58d99a8abb5e751abb6a838af6f12887b5418
Successfully built data
Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3


In [ ]:
!pip install theano
!pip install --upgrade theano
!conda install theano
!pip show theano

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for theano: filename=Theano-1.0.5-py3-none-any.whl size=2668109 sha256=405d1d81e5362316d56aa1052ea28e5291331451e1d54b1deabcff097c8ffb2b
  Stored in directory: /root/.cache/pip/wheels/d9/e6/7d/2267d21a99e4ab8276f976f293b4ff23f50c9d809f4a216ebb
Successfully built theano
/bin/bash: line 1: conda: command not found
Name: Theano
Version: 1.0.5
Summary: Optimizing compiler for evaluating mathematical expressions on CPUs and GPUs.
Home-page: http://deeplearning.net/software/theano/
Author: LISA laboratory, University of Montreal
Author-email: theano-dev@googlegroups.com
License: BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, scipy, six
Required-by: 


Paraphrase Identification between multiple text files

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import itertools

class ParaphraseDataset(Dataset):
    def __init__(self, sentence_pairs, tokenizer, max_length=128):
        self.sentence_pairs = sentence_pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentence_pairs)

    def __getitem__(self, idx):
        sentence1, sentence2 = self.sentence_pairs[idx]
        inputs = self.tokenizer(sentence1, sentence2, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'sentence1': sentence1,
            'sentence2': sentence2,
        }

class BERTForParaphraseDetection(torch.nn.Module):
    def __init__(self, pretrained_model_name):
        super(BERTForParaphraseDetection, self).__init__()
        self.model = BertForSequenceClassification.from_pretrained(pretrained_model_name)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def load_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f]

file_paths = ["/content/Text_file-1.txt", "/content/Text_file-2.txt"]

file_sentences = [load_sentences(file_path) for file_path in file_paths]

file_combinations = list(itertools.combinations(enumerate(file_sentences), 2))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BERTForParaphraseDetection('bert-base-uncased')
model.to(device)
model.eval()

similarity_threshold = 0.5

similarity_scores = {}
similar_sentence_pairs = {}  # Dictionary to store similar sentence pairs

for (idx1, sentences1), (idx2, sentences2) in file_combinations:
    sentence_pairs = [(s1, s2) for s1 in sentences1 for s2 in sentences2]

    dataset = ParaphraseDataset(sentence_pairs, tokenizer)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=False)

    paraphrase_count = 0
    total_pairs = len(sentence_pairs)

    # Initialize a list to store similar sentence pairs for the current file pair
    similar_pairs = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Evaluating files {idx1+1} and {idx2+1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            logits = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(logits, dim=1)

            # Iterate through predictions
            for i, pred in enumerate(predictions):
                if pred == 1:  # If the prediction is a paraphrase
                    paraphrase_count += 1
                    # Add similar sentence pair to the list
                    sentence1 = batch['sentence1'][i]
                    sentence2 = batch['sentence2'][i]
                    similar_pairs.append((sentence1, sentence2))

    # Calculate similarity score
    similarity_score = paraphrase_count / total_pairs
    pair_name = f"File {idx1+1} & File {idx2+1}"
    similarity_scores[pair_name] = similarity_score

    # Store the similar pairs for the current file pair
    similar_sentence_pairs[pair_name] = similar_pairs

# Print similarity scores for each file pair
print("\nSimilarity Scores for File Pairs:")
for pair, score in similarity_scores.items():
    print(f"{pair}: Similarity Score = {score:.2f}")

# Print similar sentence pairs for each file pair
print("\nSimilar Sentence Pairs for File Pairs:")
for pair_name, pairs in similar_sentence_pairs.items():
    if len(pairs) > 0:
        print(f"\n{pair_name}:")
        for sentence1, sentence2 in pairs:
            print(f"  Sentence 1: {sentence1}")
            print(f"  Sentence 2: {sentence2}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating files 1 and 2:   0%|          | 0/2 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even 


Similarity Scores for File Pairs:
File 1 & File 2: Similarity Score = 0.20

Similar Sentence Pairs for File Pairs:

File 1 & File 2:
  Sentence 1: 
  Sentence 2: 
  Sentence 1: 
  Sentence 2: 


In [ ]:
pip install python-docx

Paraphrase Identification between multiple Word documents


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import itertools
import os
from docx import Document
class ParaphraseDataset(Dataset):
    def __init__(self, sentence_pairs, tokenizer, max_length=128):
        self.sentence_pairs = sentence_pairs
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.sentence_pairs)
    def __getitem__(self, idx):
        sentence1, sentence2 = self.sentence_pairs[idx]
        inputs = self.tokenizer(sentence1, sentence2, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'sentence1': sentence1,
            'sentence2': sentence2,
        }
class BERTForParaphraseDetection(torch.nn.Module):
    def __init__(self, pretrained_model_name):
        super(BERTForParaphraseDetection, self).__init__()
        self.model = BertForSequenceClassification.from_pretrained(pretrained_model_name)
    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    return text

file_paths = ["/content/The Taj Mahal-1.docx","/content/The Taj Mahal.docx", "/content/The Great Wall of China.docx"]

file_sentences = [extract_text_from_docx(file_path) for file_path in file_paths]

file_combinations = list(itertools.combinations(enumerate(file_sentences), 2))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BERTForParaphraseDetection('bert-base-uncased')
model.to(device)
model.eval()

similarity_threshold = 0.5
similarity_scores = {}
similar_sentence_pairs = {}
for (idx1, sentences1), (idx2, sentences2) in file_combinations:
    sentence_pairs = [(s1, s2) for s1 in sentences1 for s2 in sentences2]
    dataset = ParaphraseDataset(sentence_pairs, tokenizer)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=False)
    paraphrase_count = 0
    total_pairs = len(sentence_pairs)
    similar_pairs = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Evaluating files {idx1+1} and {idx2+1}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            logits = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(logits, dim=1)

            for i, pred in enumerate(predictions):
                if pred == 1:
                    paraphrase_count += 1
                    sentence1 = batch['sentence1'][i]
                    sentence2 = batch['sentence2'][i]
                    similar_pairs.append((sentence1, sentence2))
    similarity_score = paraphrase_count / total_pairs
    pair_name = f"File {idx1+1} & File {idx2+1}"
    similarity_scores[pair_name] = similarity_score
    similar_sentence_pairs[pair_name] = similar_pairs
print("\nSimilarity Scores for File Pairs:")
for pair, score in similarity_scores.items():
    print(f"{pair}: Similarity Score = {score:.2f}")
print("\nSimilar Sentence Pairs for File Pairs:")
for pair_name, pairs in similar_sentence_pairs.items():
    if len(pairs) > 0:
        print(f"\n{pair_name}:")
        for sentence1, sentence2 in pairs:
            print(f"  Sentence 1: {sentence1}")
            print(f"  Sentence 2: {sentence2}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating files 1 and 2:   0%|          | 0/2 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even 


Similarity Scores for File Pairs:
File 1 & File 2: Similarity Score = 0.00
File 1 & File 3: Similarity Score = 0.08
File 2 & File 3: Similarity Score = 0.11

Similar Sentence Pairs for File Pairs:

File 1 & File 3:
  Sentence 1: The Taj Mahal, one of the most iconic landmarks in the world, is located in Agra, India. Commissioned in 1632 by the Mughal emperor Shah Jahan in memory of his beloved wife Mumtaz Mahal, the monument stands as a testament to eternal love and exquisite craftsmanship. It took over 20,000 artisans from across the empire and beyond to construct the Taj Mahal, with construction spanning over two decades, completing in 1653. Its stunning white marble facade is adorned with intricate carvings and inlaid with semi-precious stones, showcasing the finest examples of Mughal architecture.
  Sentence 2: The Great Wall of China is an iconic symbol of China's rich history and engineering prowess. Stretching over 13,000 miles across northern China, it is one of the most impre